<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br><br><br>

# Listed Volatility and Variance Derivatives

**Wiley Finance (2017)**

Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [@dyjh](http://twitter.com/dyjh) | http://books.tpq.io

<img src="https://hilpisch.com/images/lvvd_cover.png" alt="Listed Volatility and Variance Derivatives" width="30%" align="left" border="0">

# DX Analytics &mdash; An Overview

## Introduction

Although Python arguably is a good programming language and ecosystem for financial analytics (see chapter _Introduction_ or chapter 1 of Hilpisch (2018)), dedicated libraries for finance are not that common. This is even more true when it comes to derivatives analytics as a sub-discipline. One exception in this regard is DX Analytics (the "dx library"), written by the  author of this book, which has a major focus on advanced derivatives and risk analytics. The central resource to get started with the library is the Web site http://dx-analytics.com.

This chapter gives an overview of the relevant parts of the library for the purposes of the then following case studies. The development of the library is guided by two central principles.

* **global valuation approach**: in practice, this approach translates into the non-redundant modeling of all risk factors (e.g. option underlyings like equity indexes) and the valuation of all derivative instruments by a unique, consistent numerical method &mdash; which is Monte Carlo simulation in the case of DX Analytics
* **unlimited computing resources**: Monte Carlo simulation is compute and memory intensive and has therefore often been dismissed as an adequate numerical method to implement, for example, front-office analytics libraries; in 2016, the technical infrastructures available to even smaller players in the financial industry have reached performance levels that 10 years ago seemed impossible or at least financially not feasible; in that sense "unlimited resources" is not to be understood literally but rather as the guiding principle that hardware and computing resources generally are no longer a bottleneck

Among others, DX Analytics provides the following features:

* **models**: models for risk factors include simple ones like geometric Brownian motion as well as more sophisticated ones like stochastic volatility jump diffusion models
* **derivatives**: derivatives models include single risk factor as well as multi risk factor models, both with European and American (Bermudan) exercise
* **portfolios**: derivatives portfolios can be arbitraritly complex with multiple, correlated risk factors and multiple, diverse derivative instruments (single risk and multi risk); simulations and valuations are implemented in a fashion that both value and risk aggregations are consistent for each Monte Carlo path

The subsequent sections illustrate the use of the library by the means of a simple, yet still realistic example with 2 correlated risk factors and 2 different options.

## Modeling Risk Factors

First, some necessary imports and in particular the import of the `dx` library.

In [ ]:
import dx
import numpy as np
np.random.seed(1000)
import pandas as pd
import datetime as dt
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

Usually, the first step is to define a model for the _risk-neutral discounting_ since all valuations are based on the risk-neutral (or martingale) pricing approach (cf. Björk (2009)). Throughout, we will work with a constant short rate model although DX Analytics also provides deterministic yield curve and stochastic short rate models.

In [ ]:
r = dx.constant_short_rate('r', 0.01)

The next step is to define a _market environment_ containing the parameter specifications needed. Several objects might have different market environments but they might also share market environments. The first risk factor to be modeled is a _geometric Brownian motion_ (Black-Scholes-Merton (1973) model). The following market environment object contains all parameters needed for this model. Comments in the code explain the single elements.

In [ ]:
## instantiation of market environment object
me_1 = dx.market_environment('me', dt.datetime(2016, 1, 1))
## starting value of simulated processes
me_1.add_constant('initial_value', 100.)
## volatiltiy factor
me_1.add_constant('volatility', 0.2)
## horizon for simulation
me_1.add_constant('final_date', dt.datetime(2016, 6, 30))
## currency of instrument
me_1.add_constant('currency', 'EUR')
## frequency for discretization (here: weekly)
me_1.add_constant('frequency', 'W')
## number of paths
me_1.add_constant('paths', 25000)
## short rate model for discount curve
me_1.add_curve('discount_curve', r)

Equipped with this object, the model object for the risk factor can be instantiated.

In [ ]:
gbm_1 = dx.geometric_brownian_motion('gbm_1', me_1)
gbm_1

The `get_instrument_values()` method initiates a Monte Carlo simulation and delivers back the simulated paths &mdash; given the parametrizations from the market environment object &mdash; as a NumPy `ndarray` object.

In [ ]:
gbm_1.get_instrument_values()

Via the `time_grid` attribute one can access the date-time information for the time series data.

In [ ]:
gbm_1.time_grid[:10]

Combining both arrays to a single pandas ``DataFrame`` object makes plotting straightforward (see the following figure).

In [ ]:
pdf_1 = pd.DataFrame(gbm_1.get_instrument_values(), index=gbm_1.time_grid)
pdf_1.iloc[:, :10].plot(legend=False, figsize=(10, 6));

<p style="font-family: monospace;">Simulated paths for the risk factor based on geometric Brownian motion

Next, we define a second risk factor, again based on geometric Brownian motion. We use the market envrionment information from the first risk factor and only overwrite the volatility value.

In [ ]:
## instantiate new market environment object
me_2 = dx.market_environment('me_2', me_1.pricing_date)
## add complete environment
me_2.add_environment(me_1)
## overwrite volatility value
me_2.add_constant('volatility', 0.5)

With this, define the second risk factor as follows.

In [ ]:
gbm_2 = dx.geometric_brownian_motion('gbm_2', me_2)

The plot in the following figure illustrates the higher volatility of the second risk factor graphically.

In [ ]:
pdf_2 = pd.DataFrame(gbm_2.get_instrument_values(), index=gbm_2.time_grid)
ax = pdf_1.iloc[:, :10].plot(legend=False, figsize=(10, 6), style=11 * ['b-']);
pdf_2.iloc[:, :10].plot(legend=False, style=11 * ['r--'], ax=ax);

<p style="font-family: monospace;">Simulated paths for the two risk factors; solid lines = low volatility, dashed lines = high volatility

## Modeling Derivatives

Based on the risk factors, we can define derivatives models for valuation. To this end, we need to add at least one (the `maturity`), in general two (`maturity` and `strike`), parameters to the market environment(s).

In [ ]:
## instantiation of market environment object for option
me_opt = dx.market_environment('me_opt', me_1.pricing_date)
## add complete market environment
me_opt.add_environment(me_1)
## add maturity date for option
me_opt.add_constant('maturity', dt.datetime(2016, 6, 30))
## add strike for option
me_opt.add_constant('strike', 110.)

The first derivative is an _American put option_ on the first risk factor `gbm_1`.

In [ ]:
am_put = dx.valuation_mcs_american_single(
            name='am_put',  # name of the option as string
            underlying=gbm_1,  # the risk factor object
            mar_env=me_opt,  # the market environment
            payoff_func='np.maximum(strike - instrument_values, 0)')

Let us calculate a Monte Carlo present value estimate and estimates for the Greeks of the American put.

In [ ]:
am_put.present_value()  # Monte Carlo estimator

In [ ]:
am_put.delta()  # delta of the option

In [ ]:
am_put.gamma()  # gamma of the option

In [ ]:
0.5 * am_put.gamma() * am_put.underlying.initial_value ** 2  # dollar gamma

In [ ]:
am_put.vega()  # vega of the option

In [ ]:
am_put.theta()  # theta of the option 

In [ ]:
am_put.rho()  # rho of the option

The second derivative is a European call option on the second risk factor `gbm_2`. It has the same strike and maturity as the American put option.

In [ ]:
eur_call = dx.valuation_mcs_european_single(
            name='eur_call',
            underlying=gbm_2,
            mar_env=me_opt,
            payoff_func='np.maximum(maturity_value - strike, 0)')

The major statistics for this option are:

In [ ]:
eur_call.present_value()

In [ ]:
eur_call.delta()

In [ ]:
eur_call.gamma()

In [ ]:
0.5 * eur_call.gamma() * eur_call.underlying.initial_value ** 2

In [ ]:
eur_call.vega()

In [ ]:
eur_call.theta()

In [ ]:
eur_call.rho()

Note that all these values might vary to a smaller or greater extent with the parameters chosen for the Monte Carlo simulation.

To conclude this section, let us analyze the European call option in a bit more detail. We want to estimate and collect the Greeks for different strikes. The following code implements the necessary steps.

In [ ]:
k_list = np.arange(80., 120.5, 2.5)
pv = []; de = []; ve = []; th = []; rh = []; ga = []
for k in k_list:
    eur_call.update(strike=k)
    pv.append(eur_call.present_value())
    de.append(eur_call.delta())
    ve.append(eur_call.vega())
    th.append(eur_call.theta())
    rh.append(eur_call.rho())
    ga.append(eur_call.gamma())

The following figure shows the results graphically.

In [ ]:
dx.plot_option_stats_full(k_list, pv, de, ga, ve, th, rh)

<p style="font-family: monospace;">Greeks of the European call option for different strikes.

## Derivatives Portfolios

The previous sections show how convenient and flexible it is to model single derivatives with DX Analytics. The numerical methods used and the API of the library mimic the working with the Black-Scholes-Merton closed option pricing formula when it comes to Greeks although the derivative itself might be much more complex than a plain vanilla European call or put option (e.g. it might have American exercise and an exotic payoff). However, the area which differentiates DX Analytics most from other derivatives analytics libraries is the global valuation approach for derivatives portfolios. How it works is explained in this section.

### Modeling Portfolios

In a *portfolio context*, we need to add information about the model class(es) to be used by the market environments of the risk factors.

In [ ]:
me_1.add_constant('model', 'gbm')
me_2.add_constant('model', 'gbm')

To compose a portfolio consisting of the two options, we need to define _derivatives positions_. Note that this step is independent from the risk factor model and option model definitions. We only use the market environment data and some additional information needed (e.g. payoff functions).

In [ ]:
put = dx.derivatives_position(
    name='put',  # name as string
    quantity=2,  # number of options in the portfolio
    underlyings=['gbm_1'],  # the underlying(s) as list object
    mar_env=me_opt,  # the market environment object
    otype='American single',  # the option type
    payoff_func='np.maximum(strike - instrument_values, 0)')  # the payoff

call = dx.derivatives_position(
    name='call',
    quantity=3,
    underlyings=['gbm_2'],
    mar_env=me_opt,
    otype='European single',
    payoff_func='np.maximum(maturity_value - strike, 0)')

In a portfolio context, we also need to define the _market_. It consists of the risk factors, the correlation between them, the derivatives positions as well as the valuation environment.

In [ ]:
risk_factors = {'gbm_1': me_1, 'gbm_2' : me_2}  # as dictionary
correlations = [['gbm_1', 'gbm_2', -0.4]]  # as list
positions = {'put' : put, 'call' : call}  # as dictionary

The *valuation environenment* (technically another market environment) contains all those parameters shared by all derivatives positions. This might imply that certain parameters from the market environments of the derivatives get replaced for the portfolio simulations and valuations. 

In [ ]:
val_env = dx.market_environment('general', dt.datetime(2016, 1, 1))
val_env.add_constant('frequency', 'W')
val_env.add_constant('paths', 25000)
val_env.add_constant('starting_date', val_env.pricing_date)
val_env.add_constant('final_date', val_env.pricing_date)
val_env.add_curve('discount_curve', r)

These new objects are needed to instantiate a `portfolio` object.

In [ ]:
port = dx.derivatives_portfolio(
        name='portfolio',  # name as string
        positions=positions,  # derivatives positions
        val_env=val_env,  # valuation environment
        risk_factors=risk_factors, # relevant risk factors
        correlations=correlations,  # correlation between risk factors
        parallel=False)  # parallel valuation True/False

### Simulation and Valuation

Simulation and valuation are now as straightforward as in the single option case.

In [ ]:
port.get_values()  # get all present values

In [ ]:
port.get_statistics()  # get major statistics

### Risk Reports

A strength of DX Analytics and the global valuation approach  is that one can easily generate consistent risk reports. By this we mean that single parameters are shocked and the effect on the portfolio value is estimated. Think of a larger portfolio containing multiple options on the S&P 500 equity index. By changing the spot value of the index, DX Analytics can estimate in a single step what the impact is on the overall portfolio value (and not only on a single option like in the case of a delta calculation).

When calling the `get_port_risk()` method you need to define which parameter shall be shocked. You get back all hypothetical portfolio values and the benchmark value without shock. The following estimates show the "portfolio deltas".

In [ ]:
deltas, benchvalue = port.get_port_risk(Greek='Delta')

In [ ]:
benchvalue

In [ ]:
deltas

There is a convenience function called `risk_report()` in DX Analytics to nicely print the results.

In [ ]:
dx.risk_report(deltas)  # gives the resulting values ...

In [ ]:
dx.risk_report(deltas.loc(axis=0)[:, 'value'] - benchvalue,
               gross=False)  # ... as net changes

"Portfolio vegas" are calculated in the same way.

In [ ]:
vegas, benchvalue = port.get_port_risk(Greek='Vega', step=0.05)

In [ ]:
dx.risk_report(vegas)

In [ ]:
dx.risk_report(vegas.loc(axis=0)[:, 'value'] - benchvalue, gross=False)

## Conclusions

This chapter provides a quick start with DX Analytics, a Python-based financial analytics library with a focus on derivatives and risk analytics. The library offers many more features than covered in this brief chapter. It is recommended to check out the main page http://dx-analytics.com and to work through the different parts of the documentation which are all based on executable Jupyter Notebooks.

The focus of this chapter is on the basic tool set and a basic understanding of the API to have a good basis for the case studies in the two subsquent chapters. The case studies use DX Analytics to model the VSTOXX volatility index by the *square-root diffusion model* from chapter _Valuing Volatility Derivatives_ as well as the *square-root jump diffusion model* from chapter _Advanced Modeling of the VSTOXX Index. The major goal of the case studies is to analyze how well the two models perform over time in replicating the market quotes of traded VSTOXX options.

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>